## 注意力机制Attention:

### 简介:

这一部分是Transformer模型的核心部分,以下部分逐步给出实现过程中可能用到的一些矩阵运算的原理， 以下代码均不需要大家实现,希望大家阅读代码以及下列文档中的信息:

https://arxiv.org/abs/1706.03762

https://jalammar.github.io/illustrated-transformer/

理解Attention的运行机制以及实现过程的数学技巧，完成最后的主文件中的HeadAttention(),MultiHeadAttention()部分。

我们虚构一组输入数据的Embedding用于这部分讲解：

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
B, T, C = 1, 8, 16   ## B: batch size 一次训练的数据量, T: context length 前文token数, C: embedding length 隐变量长度
inputData = torch.rand(size=(B,T,C))

for i in range(T):
    print(f"Embedding of {i}th position:\n {inputData[0,i]}")


Embedding of 0th position:
 tensor([0.6215, 0.8552, 0.4837, 0.7872, 0.0755, 0.2209, 0.9917, 0.1800, 0.2335,
        0.6284, 0.2222, 0.3850, 0.5906, 0.6990, 0.2941, 0.9429])
Embedding of 1th position:
 tensor([0.6087, 0.8247, 0.8361, 0.2070, 0.5286, 0.6780, 0.9830, 0.4436, 0.3387,
        0.0432, 0.0465, 0.5585, 0.9630, 0.4677, 0.7098, 0.9967])
Embedding of 2th position:
 tensor([0.8664, 0.4121, 0.4108, 0.7874, 0.9328, 0.6227, 0.9799, 0.6877, 0.9111,
        0.2651, 0.5743, 0.6642, 0.0705, 0.9647, 0.8043, 0.0985])
Embedding of 3th position:
 tensor([0.1162, 0.8443, 0.8840, 0.9209, 0.5907, 0.9122, 0.9107, 0.5899, 0.8703,
        0.9601, 0.5479, 0.4325, 0.3125, 0.4490, 0.7440, 0.7145])
Embedding of 4th position:
 tensor([0.6183, 0.8303, 0.9758, 0.5148, 0.7015, 0.0972, 0.5896, 0.3761, 0.2082,
        0.8351, 0.7580, 0.0576, 0.5821, 0.9855, 0.9663, 0.2860])
Embedding of 5th position:
 tensor([0.6102, 0.6918, 0.5958, 0.1962, 0.1125, 0.7893, 0.9835, 0.7357, 0.6670,
        0.5160, 0.2809, 0.8

Attention从直观上可以理解为对前文各个位置信息的融合以获得当前语境所需的信息。 一个最简单的融合方式为对前文Embedding加权求和作为当前位置的信息。

我们计算第i个位置的融合后的embedding:

假设前i个位置的embedding的权重相同，均为1/i，即更新后第i个位置embedding为前文所有位置embedding的平均值：

In [3]:
def Attention_version1(contextEmbeddings):
    for i in range(T):
        context_embeddings = contextEmbeddings[0,:i+1,:] ## shape [i+1, C]
        new_embedding_for_i = torch.mean(context_embeddings,dim=0)
        contextEmbeddings[0,i] = new_embedding_for_i
    return contextEmbeddings

print("Embedding of Data after aggregate context embedding:\n", Attention_version1(inputData))

Embedding of Data after aggregate context embedding:
 tensor([[[0.6215, 0.8552, 0.4837, 0.7872, 0.0755, 0.2209, 0.9917, 0.1800,
          0.2335, 0.6284, 0.2222, 0.3850, 0.5906, 0.6990, 0.2941, 0.9429],
         [0.6151, 0.8399, 0.6599, 0.4971, 0.3020, 0.4494, 0.9874, 0.3118,
          0.2861, 0.3358, 0.1343, 0.4717, 0.7768, 0.5834, 0.5020, 0.9698],
         [0.7010, 0.7024, 0.5181, 0.6905, 0.4368, 0.4310, 0.9863, 0.3932,
          0.4769, 0.4098, 0.3103, 0.5070, 0.4793, 0.7490, 0.5335, 0.6704],
         [0.5134, 0.8105, 0.6364, 0.7239, 0.3512, 0.5034, 0.9690, 0.3687,
          0.4667, 0.5835, 0.3037, 0.4491, 0.5398, 0.6201, 0.5184, 0.8244],
         [0.6139, 0.8077, 0.6548, 0.6427, 0.3734, 0.3404, 0.9048, 0.3260,
          0.3343, 0.5585, 0.3457, 0.3741, 0.5937, 0.7274, 0.5629, 0.7387],
         [0.6125, 0.7846, 0.5915, 0.5896, 0.2752, 0.4557, 0.9705, 0.3859,
          0.4107, 0.5053, 0.2662, 0.4981, 0.5937, 0.7068, 0.5498, 0.8432],
         [0.6080, 0.7707, 0.5442, 0.6837, 0.3367, 0.

我们将上述的mean操作换为等价的矩阵运算，以i=3 为例：

new_embedding_for_3 = torch.mean(contextEmbeddings[0,:3+1],dim=0)

等价于(@ 是矩阵乘法):

new_embedding_for_3 = contextEmbeddings[0] @ torch.tensor([1/4,1/4,1/4,1/4,0,0,0,0])

In [4]:
def Attention_version2(contextEmbeddings):
    for i in range(T):
        weight = torch.cat((torch.ones(i+1) / (i+1),torch.zeros(T-i-1,dtype=torch.float)),dim=0)
        contextEmbeddings[0,i] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version2: ",torch.all(Attention_version1(inputData) == Attention_version2(inputData)).item())

Attention_version1 equivalent to Attention_version2:  True


接下来我们用矩阵运算进一步简化上述运算，移除其中的for循环:

其中 weight = torch.tril(torch.ones(T,T)) 得到:

[[1., 0., 0., 0., 0., 0., 0., 0.],

 [1., 1., 0., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 1.]]
 
表示前文的求和权重相同都为一。

weight = weight.masked_fill(weight==0,float("-inf"))

weight = F.softmax(weight)

这两行用于归一化weight,即每一次加权求和的权重和为1，具体详见Softmax公式,我们可得到：

[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],

[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],

[0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],

[0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]


In [5]:
def Attention_version3(contextEmbeddings):
    B, T, C = contextEmbeddings.shape
    weight = torch.tril(torch.ones(T,T))
    print("weight of context embeddings:\n",weight)
    weight = weight.masked_fill(weight==0,float("-inf"))
    weight = F.softmax(weight,dim=1)
    print("weight of context embeddings after regularization:\n",weight)
    contextEmbeddings[0] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version3: ",torch.all(Attention_version1(inputData) == Attention_version3(inputData)).item())

weight of context embeddings:
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
weight of context embeddings after regularization:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.125

最后，我们确定计算weight的方法，上述三个版本的weight都是假定所有前文信息的重要程度相同,在大语言模型中，我们希望有一个灵活的方式计算前文信息对应当前语境的重要程度，为此Transformer引入了Query，Key，Value:

其中Query可以理解为当前语境对于前文信息的需求，Key可以理解为前文包含信息的索引，Value为前文所包含的信息。

Query 和 Key 用来计算信息融合的weight.

如何计算Query和Key，并用他们计算weight对Value加权求和是这次实验的重点内容，这里不能给出大家具体代码，希望大家参见Attention is All you need原论文以及助教提供的文档最后的参考链接学习这部分。

利于Query和Key得出的是信息相关性，我们需要遮盖住下文的信息(生成第i个token时，只可以使用0到i-1处的信息)，并且要对相关性归一化使之可以作为weight。这里利于Attension_version3()中的结论给出如何对计算出来的相关性加掩码和归一化:


In [6]:
def weight_mask_and_normalization(weight):
    tril = torch.tril(torch.ones_like(weight))
    weight = weight.masked_fill(tril == 0, float("-inf"))
    weight = F.softmax(weight,dim=-1)
    return weight

weight = torch.rand(T,T)
print("weight before mask and normalize:\n",weight)
print("weight after mask and normalize:\n",weight_mask_and_normalization(weight))

weight before mask and normalize:
 tensor([[0.8523, 0.4282, 0.0460, 1.0000, 0.3809, 0.7626, 0.7077, 0.6811],
        [0.6787, 0.4468, 0.4300, 0.7270, 0.3811, 0.0037, 0.3554, 0.8331],
        [0.1003, 0.8613, 0.4618, 0.3347, 0.1696, 0.4406, 0.1559, 0.4696],
        [0.4371, 0.6387, 0.4195, 0.6367, 0.2242, 0.1279, 0.5870, 0.4034],
        [0.5012, 0.3698, 0.8895, 0.3906, 0.5233, 0.1319, 0.0744, 0.3036],
        [0.7953, 0.4295, 0.5013, 0.7761, 0.8805, 0.9213, 0.9184, 0.9161],
        [0.4488, 0.5384, 0.0650, 0.3066, 0.6172, 0.4112, 0.9485, 0.9554],
        [0.1669, 0.7973, 0.2770, 0.7504, 0.0314, 0.5796, 0.3120, 0.2000]])
weight after mask and normalize:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5577, 0.4423, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2185, 0.4678, 0.3137, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2259, 0.2763, 0.2220, 0.2758, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1898, 0.1664, 0.2798, 0.1699, 0